# Vector Similarity

### Importing the required libraries

In [ ]:
import io
import numpy as np
# from google_trans_new import google_translator 
from scipy.spatial.distance import cosine
import json 
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import word_tokenize
from itertools import groupby 
!pip install -U torch
import torch
!pip install -U transformers
!pip install -U sentence-transformers
import transformers


In [1]:
from sentence_transformers import SentenceTransformer,util
import numpy as np

In [ ]:
import gensim,nltk,warnings
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize,word_tokenize
warnings.filterwarnings(action = 'ignore')
from os import listdir
from os.path import isfile, join
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
import re
!pip install sent2vec
!pip install transformers
from scipy import spatial
from sent2vec.vectorizer import Vectorizer
import numpy as np
import torch
import transformers
from transformers import BertTokenizer
from transformers import BertModel

In [4]:
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#@title
def embedsentence(text,model,tokenizer):
  # Add the special tokens.
  marked_text = "[CLS] " + text + " [SEP]"

  # Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)

  # Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  # Mark each of the 22 tokens as belonging to sentence "1".
  segments_ids = [1] * len(tokenized_text)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  # Run the text through BERT, and collect all of the hidden states produced
  # from all 12 layers. 
  with torch.no_grad():

      outputs = model(tokens_tensor, segments_tensors)
      # Evaluating the model will return a different number of objects based on 
      # how it's  configured in the `from_pretrained` call earlier. In this case, 
      # becase we set `output_hidden_states = True`, the third item will be the 
      # hidden states from all layers. See the documentation for more details:
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2]
  # Concatenate the tensors for all layers. We use `stack` here to
  # create a new dimension in the tensor.
  token_embeddings = torch.stack(hidden_states, dim=0)

  # Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  # Swap dimensions 0 and 1.
  token_embeddings = token_embeddings.permute(1,0,2)

  # `hidden_states` has shape [13 x 1 x 22 x 768]

  # `token_vecs` is a tensor with shape [22 x 768]
  token_vecs = hidden_states[-2][0]

  # Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)
  return sentence_embedding

In [6]:
bert_model = BertModel.from_pretrained(pretrained_model_name_or_path= 'DeepPavlov/bert-base-multilingual-cased-sentence')
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path= 'DeepPavlov/bert-base-multilingual-cased-sentence')

In [10]:
pt_batch = tokenizer(
    "मैं बहुत ख़ुश हूँ",
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt")
outputs = bert_model(**pt_batch)
pooled_output = outputs[1]

In [43]:
sentence1 = "Dog"
sentence2 = "मुझे कुत्ते अच्छे लगते हैं"
# encode sentences to get their embeddings
embedding1 = embedsentence(sentence1,bert_model,tokenizer)
embedding2 = embedsentence(sentence2,bert_model,tokenizer)
# compute similarity scores of two embeddings
cosine_scores = 1 - spatial.distance.cosine(embedding1, embedding2)
print("Sentence 1:", sentence1)
print("Sentence 2:", sentence2)
print("Similarity score:", cosine_scores)

Sentence 1: Dog killed him
Sentence 2: मुझे कुत्ते अच्छे लगते हैं
Similarity score: 0.8705136179924011


### Loading data from json file

In [7]:
import json 
# Opening JSON file 
f = open('/content/drive/MyDrive/test_data_v2.json',encoding = 'utf-8') 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 
  
# Iterating through the json 
# list 
d = {}
d = data
  
# Closing file 
f.close()

In [8]:
### Getting a_c_p for getting all the triples
# Opening JSON file 
f = open('/content/drive/MyDrive/a_c_p.json',) 
data = json.load(f) 
acp = {}
acp = data
f.close() 

### Storing sentences and triples from one article
Here we store all the sentences and the triples from one article. The article is about the actress 'Kalpana'. We display 5 sentences and 5 triples corresponding to her here.

### Obtaining sentence and triple embeddings
We obtain the sentence embeddings by taking each sentence, obtaining word embeddings for each word in the sentence, and averaging the word embeddings. For triple embeddings , we average the embeddings for each word in the triple.

In [9]:
# Matching Triples with sentences
actors_test, cricketers_test, politicians_test = d['actors'], d['cricketers'], d['politicians']
actors, cricketers, politicians = actors_test, cricketers_test, politicians_test

# Creating the dictionary for the test data where key = sentence and value = list of matching triples
actors_test_dict, cricketers_test_dict, politicians_test_dict = {}, {}, {}
for l in actors_test:
    for k, v in l.items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    actors_test_dict[sentence] = set(t)

for l in cricketers_test:
    for k, v in l.items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    cricketers_test_dict[sentence] = set(t)

for l in politicians_test:
    for k, v in l.items():
        if k == 'sentence':
            sentence = v
        if k == 'triples':
            triple_list = v
    t = [(e['subject'], e['predicate'], e['object']) for e in triple_list]
    politicians_test_dict[sentence] = set(t)
    
len(actors_test_dict),len(cricketers_test_dict),len(politicians_test_dict)

(60, 13, 30)

In [10]:
# We had got a_c_p.json. We keep only the relevant triples by filtering by entity id in test annotated data
actors_trip, cricketers_trip, politician_trip = [], [], []
actors_sent, cricketers_sent, politician_sent = [], [], []

# Putting actors,cricketers and politicians from a_c_p
act, cric, pol = acp['a'], acp['c'], acp['p']

############### Actors #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in actors_test:
    eid = e['entity_id']
    for ele in act:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = act[ele]['triples']
            subject = act[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            actors_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in actors_test:
        if eid == e['entity_id']:
            sentence = e['sentence']
            sentence_list.append(sentence)
    actors_sent.append(sentence_list)


############### Cricketers #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in cricketers_test:
    eid = e['entity_id']
    for ele in cric:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = cric[ele]['triples']
            subject = cric[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            cricketers_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in cricketers_test:
        if eid == e['entity_id']:
            sentence = e['sentence']
            sentence_list.append(sentence)
    cricketers_sent.append(sentence_list)


############### Politicians #######################

# Iterating over test annotated data and keeping relevant triples only
entity_tracking = []
for e in politicians_test:
    eid = e['entity_id']
    for ele in pol:
        if eid == ele and eid not in entity_tracking:
            entity_tracking.append(eid)
            # Getting triples for the matching entity id
            triples = pol[ele]['triples']
            subject = pol[ele]['personLabel']
            triplist = []
            for trip in triples:
                predicate = trip['propertyLabel']
                obj = trip['objectLabel']
                trip_tuple = (subject, predicate, obj)
                triplist.append(trip_tuple)
            politician_trip.append(triplist)

# Iterating over test annotated data and grouping annotated sentences together by entity id

for eid in entity_tracking:
    sentence_list = []
    for e in politicians_test:
        if eid == e['entity_id']:
            sentence = e['sentence']
            sentence_list.append(sentence)
    politician_sent.append(sentence_list)

In [11]:
len(actors_sent), len(cricketers_sent),len(politician_sent)

(34, 13, 27)

In [12]:
len(actors_trip), len(cricketers_trip),len(politician_trip)

(34, 13, 27)

In [13]:
actors_trip[0]

[('Karan Tacker',
  'image',
  'http://commons.wikimedia.org/wiki/Special:FilePath/Karan%20Tacker%20at%20the%2011th%20Star%20Parivaar%20Awards%202013.jpg'),
 ('Karan Tacker', 'place of birth', 'Mumbai'),
 ('Karan Tacker', 'sex or gender', 'male'),
 ('Karan Tacker', 'country of citizenship', 'India'),
 ('Karan Tacker', 'instance of', 'human'),
 ('Karan Tacker', 'occupation', 'actor'),
 ('Karan Tacker', 'occupation', 'television actor'),
 ('Karan Tacker', 'occupation', 'film actor'),
 ('Karan Tacker', 'Commons category', 'Karan Tacker'),
 ('Karan Tacker', 'date of birth', '1986-05-11T00:00:00Z'),
 ('Karan Tacker', 'given name', 'Karan'),
 ('Karan Tacker', 'Twitter username', 'karantacker'),
 ('Karan Tacker', 'Instagram username', 'karantacker'),
 ('Karan Tacker', 'Facebook ID', 'tackerkaran'),
 ('Karan Tacker', 'social media followers', '1067819')]

In [34]:
def matches(sentences,triples,model,tokenizer):       # Input is a sentence list and triple list for one article
    matches_dict = {}                 # Dictionary to store the sentences with the matching triples. key = sent, value = triple
    for sent in sentences:            # For each sent in sentences
        ent_matchlist = []
        pt_batch = tokenizer(
                              sent,
                              padding=True,
                              truncation=True,
                              max_length=512,
                              return_tensors="pt")
        outputs = model(**pt_batch)
        pooled_output_sentence = outputs[1]
        
        for ent in triples:
            pred = ent[1]
            obj = ent[2]
            # print(pred)
            pt_batch = tokenizer(
                              pred + " " + obj,
                              padding=True,
                              truncation=True,
                              max_length=512,
                              return_tensors="pt")
            outputs = model(**pt_batch)
            pooled_output_triples = outputs[1]
            cosine_scores = util.pytorch_cos_sim(pooled_output_sentence, pooled_output_triples)
            similarity = cosine_scores.item()
#             ent = list(ent)
            if similarity > 0.75:
                ent_matchlist.append(ent)
        if len(ent_matchlist)>0:    
            matches_dict[sent] = set(ent_matchlist)
    return matches_dict

In [35]:
matches_act = {}
for sent_list, triple_list in tqdm(zip(actors_sent, actors_trip)):
    for k, v in matches(sent_list, triple_list,bert_model,tokenizer).items():
        matches_act[k] = v

matches_cric = {}
for sent_list, triple_list in tqdm(zip(cricketers_sent, cricketers_trip)):
    for k, v in matches(sent_list, triple_list,bert_model,tokenizer).items():
        matches_cric[k] = v

matches_pol = {}
for sent_list, triple_list in tqdm(zip(politician_sent, politician_trip)):
    for k, v in matches(sent_list, triple_list,bert_model,tokenizer).items():
        matches_pol[k] = v

34it [02:29,  4.41s/it]
13it [00:20,  1.59s/it]
27it [01:11,  2.67s/it]


### Evaluation : Precision and Recall

In [36]:
def evaluate(test_dict, matches_dict):
    sum_prec = 0
    for key, val in matches_dict.items():
        tp, fp = 0, 0
        for k, v in test_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        if (tp+fp) != 0:
            prec = tp/(tp + fp)
        else:
            prec = 0
        sum_prec = prec + sum_prec

    sum_rec = 0
    for k, v in test_dict.items():
        rec = 0
        tp, fp = 0, 0
        for key, val in matches_dict.items():
            # If sentence matches
            if k == key:
                for ent in v:
                    for trip in val:
                        if ent == trip:
                            tp = tp + 1
                for trip in val:
                    flag = 0
                    for ent in v:
                        if ent == trip:
                            flag = 1
                            break
                    if flag == 0:
                        fp = fp + 1
                break
        rec = tp/len(v)
        sum_rec = rec + sum_rec

    avg_rec, avg_prec = sum_rec/len(test_dict), sum_prec/len(matches_dict)
    return avg_rec, avg_prec


avg_rec_act, avg_prec_act = evaluate(actors_test_dict, matches_act)
avg_rec_cric, avg_prec_cric = evaluate(cricketers_test_dict, matches_cric)
avg_rec_pol, avg_prec_pol = evaluate(politicians_test_dict, matches_pol)

In [37]:
(avg_rec_act,avg_prec_act), (avg_rec_cric, avg_prec_cric) , (avg_rec_pol, avg_prec_pol)

((0.3634920634920634, 0.1724109779109779),
 (0.43589743589743585, 0.2992724867724868),
 (0.4222222222222222, 0.1695445893975306))

In [38]:
AverageRecall = (avg_rec_act + avg_rec_cric + avg_rec_pol)/3
AveragePrecision = (avg_prec_act + avg_prec_cric + avg_prec_pol)/3

AverageRecall, AveragePrecision

(0.40720390720390715, 0.21374268469366509)

### Below we have returned the top triple matches for each hindi sentence in the article

In [19]:
matches_cric

{'अमय खुरासिया एक पूर्व भारतीय क्रिकेटर हैं।': {('Amay Khurasiya',
   'CricketArchive player ID',
   '7484'),
  ('Amay Khurasiya', 'ESPNcricinfo.com player ID', '30112'),
  ('Amay Khurasiya', 'Freebase ID', '/m/08n86s'),
  ('Amay Khurasiya', 'Quora topic ID', 'Amay-Khurasiya'),
  ('Amay Khurasiya', 'country for sport', 'India'),
  ('Amay Khurasiya', 'country of citizenship', 'India'),
  ('Amay Khurasiya', 'date of birth', '1972-05-18T00:00:00Z'),
  ('Amay Khurasiya', 'instance of', 'human'),
  ('Amay Khurasiya', 'member of sports team', 'India national cricket team'),
  ('Amay Khurasiya', 'member of sports team', 'Madhya Pradesh cricket team'),
  ('Amay Khurasiya', 'occupation', 'cricketer'),
  ('Amay Khurasiya', 'place of birth', 'Jabalpur'),
  ('Amay Khurasiya', 'sex or gender', 'male'),
  ('Amay Khurasiya', 'sport', 'cricket')},
 'कुमार श्री इंद्रजीत सिंहजी माधवसिंहजी (pronunciation सहायता·सूचना) (१५ जून १९३७ - १२ मार्च २०११) एक भारतीय क्रिकेट टीम के टेस्ट क्रिकेट खिलाड़ी थे ': {('K

In [20]:
cricketers_test_dict

{'अमय खुरासिया एक पूर्व भारतीय क्रिकेटर हैं।': {('Amay Khurasiya',
   'country for sport',
   'India'),
  ('Amay Khurasiya', 'country of citizenship', 'India'),
  ('Amay Khurasiya', 'occupation', 'cricketer'),
  ('Amay Khurasiya', 'sport', 'cricket')},
 'कुमार श्री इंद्रजीत सिंहजी माधवसिंहजी (pronunciation सहायता·सूचना) (१५ जून १९३७ - १२ मार्च २०११) एक भारतीय क्रिकेट टीम के टेस्ट क्रिकेट खिलाड़ी थे ': {('Kumar Indrajitsinhji',
   'date of birth',
   '1937-07-15T00:00:00Z'),
  ('Kumar Indrajitsinhji', 'occupation', 'cricketer'),
  ('Kumar Indrajitsinhji', 'sport', 'cricket')},
 'कोट्टरी सुबन्ना नायडु भारतीय क्रिकेट खिलाड़ी थे, जिन्होंने 1934 से 1952 तक ग्यारह टेस्ट खेले।': {('C. S. Nayudu',
   'country for sport',
   'India'),
  ('C. S. Nayudu', 'occupation', 'cricketer'),
  ('C. S. Nayudu', 'sex or gender', 'male'),
  ('C. S. Nayudu', 'sport', 'cricket')},
 'गैरी विल्सन (जन्म; ५ फरवरी १९८६, डुन्डोनाल्ड) आयरलैंड क्रिकेट टीम के वर्तमान कप्तान हैं।': {('Gary Wilson',
   'date of birth',
 

#### We do find quite a few of the matching triples to be relevant to the sentence. But, there are a few irrelevant matches as well.
Upon analysis, we think the word overlap is working better than the vector similarity approach. A possible reason can be that when we simply average out the words in a sentence, and when we average out the words in the triples and then find the similarity between these two averages, some semantic information is lost. So, triples that should have been irrelevant are also found as similar. As the word overlap method is a strictly string overlap, the relevance is much stronger.